In [1]:
import itertools
import os
from functools import partial
from typing import Union, Collection, Optional

import visdom
import torch
from torch.nn import Parameter, Module
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset
import math

from tqdm.notebook import tqdm

from causalpy import SCM, LinearAssignment, NoiseGenerator, DiscreteNoise
from causalpy.neural_networks import cINN, L0InputGate
import pandas as pd
import numpy as np
from examples.simulation_linear import simulate
from causalpy.causal_prediction.interventional import ICPredictor, AgnosticPredictor
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import wasserstein_distance
from plotly import graph_objs as go
from build_scm_funcs import *
from linear_regression_eval import *


import torch as th
import math
from torch.utils.data import Sampler

/home/michael/GitHub/CausalPy/causalpy/causal_prediction/interventional/agnostic.py:324: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


In [2]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 0
np.random.seed(seed)
beta_star = None
scm_generator, target_var = build_scm_medium, "Y"
for sample_size in [30, 60, 90, 120, 200, 300, 600, 900, 1000, 1100, 1200, 1300, 1400, 1500, 3000]:
# (build_scm_large, "Y"),
# (partial(simulate, nr_genes=15), "G_12"),
# (partial(simulate, nr_genes=20), "G_16"),
# (partial(simulate, nr_genes=25), "G_21"),
# (partial(simulate, nr_genes=30), "G_29"),
    nr_repetitions = 20
    for rep in range(nr_repetitions):
        (
            complete_data,
            environments,
            scm,
            possible_parents,
            target_parents,
        ) = generate_data_from_scm(
            scm=scm_generator(seed=seed),
            markovblanket_interv_only=False,
            target_var=target_var,
            sample_size=sample_size,
            seed=seed,
        )
        target_parents_indices = np.array(
            [possible_parents.index(par) for par in target_parents]
        )
        nr_envs = np.unique(environments).max() + 1

        results = []
        epochs = 300
        use_visdom = False
        # for _ in range(nr_repetitions):
        print("Sample size:", sample_size)
        ap = AgnosticPredictor(
            epochs=epochs, batch_size=1000, visualize_with_visdom=use_visdom
        )
        results.append(ap.infer(complete_data, environments, target_var,))
        if beta_star is None:
            assignment = scm[target_var][1][scm.function_key]
            beta_star = np.array([assignment.offset] + assignment.coefficients.tolist())
        print(results[-1])
        (res_hat, beta_hat), _, y_hat = evaluate(
            complete_data,
            ap,
            environments,
            ground_truth_assignment=scm[target_var][1][scm.function_key],
            x_vars=target_parents,
            targ_var=target_var,
            plot=False
        )
        betas.append(beta_hat)
        

Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(loc=0, scale=1) ]
X_5 := f(N, X_3, Y) = 0.5 + 1 N + -0.7 X_3 + 0.4 Y	 [ N := Nor


{'X_0': 1.0, 'X_1': 1.0, 'X_2': 1.0, 'X_3': 1.0, 'X_4': 1.0, 'X_5': 1.0}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(loc=0,


{'X_0': 0.17, 'X_1': 0.17, 'X_2': 0.21, 'X_3': 0.17, 'X_4': 0.36, 'X_5': 0.24}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(


{'X_0': 1.0, 'X_1': 0.97, 'X_2': 0.96, 'X_3': 1.0, 'X_4': 1.0, 'X_5': 0.94}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(loc


{'X_0': 0.84, 'X_1': 0.83, 'X_2': 0.74, 'X_3': 0.92, 'X_4': 0.87, 'X_5': 0.82}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(

...


{'X_0': 0.92, 'X_1': 0.49, 'X_2': 0.56, 'X_3': 0.96, 'X_4': 0.74, 'X_5': 0.56}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(


{'X_0': 1.0, 'X_1': 0.51, 'X_2': 1.0, 'X_3': 1.0, 'X_4': 1.0, 'X_5': 1.0}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(loc=0


{'X_0': 1.0, 'X_1': 0.0, 'X_2': 0.0, 'X_3': 1.0, 'X_4': 1.0, 'X_5': 0.0}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(loc=0,


{'X_0': 1.0, 'X_1': 0.0, 'X_2': 0.0, 'X_3': 1.0, 'X_4': 1.0, 'X_5': 0.0}
Environment 1: Intervention on variable X_0 for value 0.
Environment 2: Intervention on variable X_1 for value 0.
Environment 3: Intervention on variable X_2 for value 0.
Environment 4: Intervention on variable X_3 for value 0.
Environment 5: Intervention on variable X_4 for value 0.
Environment 6: Intervention on variable X_5 for value 0.
Structural Causal Model of 7 variables: X_0, X_1, X_2, X_3, Y, X_4, X_5
Following variables are actively intervened on: []
Current Assignment Functions are:
X_0 := f(N) = 1 N	 [ N := Normal(loc=0, scale=1) ]
X_1 := f(N, X_0) = 1 + 1 N + 1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_2 := f(N, X_0, X_1) = 1 + 1 N + 0.8 X_0 + -1.2 X_1	 [ N := Normal(loc=0, scale=1) ]
X_3 := f(N, X_1, X_2) = 1 N + 0.3 X_1 + 0.4 X_2	 [ N := Normal(loc=0, scale=1) ]
  Y := f(N, X_3, X_0) = 0.6 + 1 N + 1 X_3 + -1 X_0	 [ N := Normal(loc=0, scale=1) ]
X_4 := f(N, Y) = 1.2 + 1 N + -0.7 Y	 [ N := Normal(loc=0,


{'X_0': 1.0, 'X_1': 0.0, 'X_2': 0.0, 'X_3': 1.0, 'X_4': 1.0, 'X_5': 0.0}
